In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from qubex.experiment import Experiment

ex = Experiment(
    chip_id="xxQ",
    qubits=[8, 10],
)

In [ ]:
result_rabi = ex.obtain_rabi_params()
result_hpi = ex.calibrate_hpi_pulse()
result_pi = ex.calibrate_pi_pulse()
result_drag_hpi = ex.calibrate_drag_hpi_pulse()
result_drag_pi = ex.calibrate_drag_pi_pulse()
result_classifier = ex.build_classifier()

In [28]:
cr_control = ex.qubit_labels[0]
cr_target = ex.qubit_labels[1]
cr_pair = (cr_control, cr_target)
cr_label = f"{cr_control}-{cr_target}"

In [ ]:
result_cr = ex.obtain_cr_params(
    *cr_pair,
    flattop_range=np.arange(0, 401, 10),
    cr_amplitude=1.0,
    cr_ramptime=50,
    n_iterations=2,
)

In [ ]:
cr_duration = 100
cr_ramptime = 40

result_cr_calib = ex.calibrate_zx90(
    *cr_pair,
    duration=cr_duration,
    ramptime=cr_ramptime,
    amplitude_range=np.linspace(0.4, 0.6, 50),
)

In [ ]:
center = result_cr_calib["calibrated_value"]

result_cr_calib = ex.calibrate_zx90(
    *cr_pair,
    duration=cr_duration,
    ramptime=cr_ramptime,
    amplitude_range=np.linspace(center - 0.1, center + 0.1, 51),
)

In [ ]:
zx90 = ex.zx90(*cr_pair)

zx90.plot()

In [ ]:
result = ex.repeat_sequence(zx90)

In [ ]:
result = ex.pulse_tomography(zx90.repeated(4), initial_state={cr_control: "0"})

In [ ]:
result = ex.pulse_tomography(zx90.repeated(4), initial_state={cr_control: "1"})

In [ ]:
result_bell = ex.measure_bell_state(*cr_pair)

In [ ]:
result_irb = ex.interleaved_randomized_benchmarking(
    target=cr_label,
    n_cliffords_range=np.arange(0, 21, 2),
    n_trials=30,
    interleaved_waveform=zx90,
    interleaved_clifford=ex.clifford["ZX90"],
)

In [ ]:
ex.save_defaults()